In [28]:
import numpy as np
import math
import cv2

In [ ]:
# cap = cv2.VideoCapture(0)

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # Convert frame to HSV for easier color detection
#     hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

#     # Define range of color to detect (e.g., blue)
#     lower_blue = np.array([110,50,50])
#     upper_blue = np.array([130,255,255])

#     # Threshold the HSV image to get only blue colors
#     mask = cv2.inRange(hsv, lower_blue, upper_blue)

#     # Find contours of the detected object
#     contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

#     for contour in contours:
#         # Optionally filter out small objects
#         area = cv2.contourArea(contour)
#         if area > 500:
#             # Calculate the center of the contour
#             M = cv2.moments(contour)
#             if M["m00"] != 0:
#                 cX = int(M["m10"] / M["m00"])
#                 cY = int(M["m01"] / M["m00"])

#                 # Mark the center on the frame
#                 cv2.circle(frame, (cX, cY), 5, (255, 255, 255), -1)

#     cv2.imshow('Frame', frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()

In [ ]:
# P1 = [xr1, yr1, 0r1] 
# P2 = [xr2, yr2, 0r2] 

In [29]:
np.zeros((800, 400))

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [30]:
robot1 = {"front":1, "back":2, "f_rgb": [136, 0, 20], "b_rgb": [3, 161, 234]}
robot2 = {"front":3, "back":4, "f_rgb": [23, 255, 22], "b_rgb": [163, 73, 164]}
ball = {"center":5, "rgb": [254, 242, 0]}

In [31]:
robots = {
    "robot1": robot1,
    "robot2": robot2
}

In [32]:

def detect_colored_dots_in_rgb(image, color_lower, color_upper):
    """
    Detects dots of a specific color in an image using RGB color space.

    :param image: The image to search in.
    :param color_lower: The lower bound of the color range in RGB.
    :param color_upper: The upper bound of the color range in RGB.
    :return: List of coordinates of detected dots.
    """
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mask = cv2.inRange(image, color_lower, color_upper)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    coordinates = []
    for contour in contours:
        M = cv2.moments(contour)
        if M['m00'] != 0:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            coordinates.append((cx, cy))

    return coordinates

# Example usage
# Load or capture an image
image = cv2.imread('Untitled.jpg')  # Replace with your image path


color_tolerance = 20



In [34]:
matrix = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

# Process each robot's front and back points
for robot, details in robots.items():
    # Detect front points
    f_lower_bound = np.array(details['f_rgb']) - color_tolerance
    f_upper_bound = np.array(details['f_rgb']) + color_tolerance
    front_coordinates = detect_colored_dots_in_rgb(image, f_lower_bound, f_upper_bound)
    
    # Mark front points on the matrix
    for coord in front_coordinates:
        matrix[coord[1], coord[0]] = details['front']
    
    # Detect back points
    b_lower_bound = np.array(details['b_rgb']) - color_tolerance
    b_upper_bound = np.array(details['b_rgb']) + color_tolerance
    back_coordinates = detect_colored_dots_in_rgb(image, b_lower_bound, b_upper_bound)
    
    # Mark back points on the matrix
    for coord in back_coordinates:
        matrix[coord[1], coord[0]] = details['back']

In [37]:
#Detect ball
lower_bound = np.array(ball['rgb']) - color_tolerance
upper_bound = np.array(ball['rgb']) + color_tolerance
ball_coordinates = detect_colored_dots_in_rgb(image, lower_bound, upper_bound)

#Mark ball points on the matrix
for coord in ball_coordinates:
    matrix[coord[1], coord[0]] = ball['center']

In [38]:
def find_coordinates(matrix, value):
    """Find the coordinates of a given value in the matrix."""
    coordinates = np.argwhere(matrix == value)
    if coordinates.size > 0:
        return coordinates[0]  # Assuming only one occurrence
    return None

def calculate_robot_position_and_orientation(matrix, front_val, back_val):
    """
    Calculate the position (x, y) and orientation (theta) of a robot.
    :param matrix: 2D numpy array representing the environment.
    :param front_val: Integer representing the front of the robot in the matrix.
    :param back_val: Integer representing the back of the robot in the matrix.
    :return: Tuple (x, y, theta) where theta is in degrees.
    """
    # Find coordinates of the front and back
    front_coords = find_coordinates(matrix, front_val)
    back_coords = find_coordinates(matrix, back_val)

    if front_coords is not None and back_coords is not None:
        # Calculate (x, y)
        x = back_coords[0]
        y = back_coords[1]
        # Calculate orientation theta
        theta = np.arctan2(front_coords[1] - back_coords[1], front_coords[0] - back_coords[0])
        theta_degrees = np.degrees(theta)

        return x, y, theta_degrees
    else:
        return None, None, None
    
    



In [51]:
# Find coordinates of the front and back
ball_coords = find_coordinates(matrix, 5)
ball_coords[0], ball_coords[1]

(397, 408)

In [39]:
r1x, r1y, r1orientation = calculate_robot_position_and_orientation(matrix, robot1.get("front"), robot1.get("back"))
r1x, r1y, r1orientation

(175, 251, -122.99770510121628)

In [40]:
r2x, r2y, r2orientation = calculate_robot_position_and_orientation(matrix, robot2.get("front"), robot2.get("back"))
r2x, r2y, r2orientation

(186, 583, 94.72798781884221)

In [52]:
distance_x = ball_coords[0] - r1x
distance_y = ball_coords[1] - r1y
distance = math.sqrt(distance_x ** 2 + distance_y ** 2)

target_angle = math.atan2(distance_y, distance_x)
angle_diff = math.degrees(target_angle - math.radians(r1orientation))

In [54]:
angle_diff, distance

(158.2659305937978, 271.9062338380641)

In [43]:
matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)